https://www.cnblogs.com/3-louise-wang/p/16671316.html
发现没有效果

In [16]:
import cv2
import numpy as np
from scipy import sparse

def getX(dst, mask, lap):
    # 1 计算求解的像素个数
    loc = np.nonzero(mask)
    num = loc[0].shape[0] 
    # 2. 有多少个像素个数则需要多少个方程，需要构造num*num大小的稀疏矩阵和num大小的b向量
    # 3. A用lilmatrix，这种矩阵方便逐个添加
    A = sparse.lil_matrix((num, num), dtype=np.float64)
    b = np.ndarray((num, ), dtype=np.float64) # 一维数组,长度只有1, 而且最外层的长度为num
    # 4. 要将每个像素映射到0~num-1的索引之中，因为A系数矩阵也是根据索引求构造的
    hhash = {(x, y): i for i, (x, y) in enumerate(zip(loc[0], loc[1]))}
    # 用于找上下左右四个像素位置
    dx = [1,0,-1,0]
    dy = [0,1,0,-1]
    # 5. 构造A系数矩阵和b向量 
    for i, (x, y) in enumerate(zip(loc[0],loc[1])):
        A[i, i] = -4
        b[i] = lap[x, y]
        p = [(x + dx[j], y + dy[j]) for j in range(4)] # p为上下左右的像素位置
        for j in range(4):
            if p[j] in hhash: # 像素位置在mask内
                A[i, hhash[p[j]]] = 1
            else:
                b[i] -= dst[p[j]]
                
    # 由于A是稀疏矩阵，可以将lilmatrix转成cscmatrix，方便矩阵运算
    A = A.tocsc()
    # 6. 求解X
    X = sparse.linalg.splu(A).solve(b)

    # 7. 将X复制到对应位置
    result = np.copy(dst)
    for i, (x, y) in enumerate(zip(loc[0],loc[1])):
        if X[i] < 0:
            X[i] = 0
        if X[i] > 255:
            X[i] = 255
        result[x, y] = X[i]

    return result

def seamlessClone(src,dst,mask,center,flag):
    # 1. 计算mask区域的外切矩形
    loc = np.nonzero(mask)
    xbegin, xend, ybegin, yend = np.min(loc[0]) - 1, np.max(loc[0]) + 1, np.min(loc[1]) - 1, np.max(loc[1]) + 1
    # 2. 根据mask区域范围截取出对应的mask、src
    cutMask = mask[xbegin:xend, ybegin:yend]
    cutSrc = src[xbegin:xend, ybegin:yend]
    # 3. 为了方便后面计算复制的位置，将mask、laplacian、src变成和dst一样大
    finalMask = np.zeros((dst.shape[0], dst.shape[1]))  # 只复制长和宽，单通道
    finalSrc = np.zeros_like(dst)
    finalLap = np.zeros_like(dst, dtype=np.float64)
    # 4. 根据中心点，得到融合的x、y范围
    xmid = (np.max(loc[0]) - np.min(loc[0])) // 2 + 1
    xbegin = center[0] - xmid
    xend = center[0] + cutMask.shape[0] - xmid
    ymid = (np.max(loc[1]) - np.min(loc[1])) // 2 + 1
    ybegin = center[1] - ymid
    yend = center[1] + cutMask.shape[1] - ymid

    # 5. 复制到对应位置
    finalMask[xbegin:xend, ybegin:yend] = cutMask
    finalSrc[xbegin:xend, ybegin:yend] = cutSrc
    
    # 6. 求融合散度
    if flag == 1:
        kernel = [np.array([[0, -1, 1]]),np.array([[1, -1, 0]]), np.array([[0], [-1], [1]]), np.array([[1], [-1], [0]])] # 四个方向梯度的过滤器，分别为x正方向、x反方向、y正方向、y反方向，散度就是在同一个方向进行两次梯度计算
        # 求src图、dst梯度
        grads = []
        for i in range(4):
            srcGrad = cv2.filter2D(np.float64(finalSrc), -1, kernel[i])
            dstGrad = cv2.filter2D(np.float64(dst), -1, kernel[i])
            grads.append(np.where(np.abs(srcGrad) > np.abs(dstGrad), srcGrad, dstGrad)) # 哪个梯度大就取哪个
        finalLap = np.sum(grads, axis=0) # 四个梯度求和，即两个散度求和，得到最终的散度

    # 7. 逐通道求解
    result = [getX(a, finalMask, b) for a, b in zip(cv2.split(dst), cv2.split(finalLap))]
    
    # 8. 合并三个通道
    final = cv2.merge(result)
    return final


# 读取前景图像
src = cv2.imread('./source.png')


# 读取背景图像
dst = cv2.imread('./target.png')

# 读取设置好的mask
mask = cv2.imread('./mask_.png')

print(src.shape, dst.shape, mask.shape)

# 设置obj拷贝到dst的位置（中心像素）
# 要填充的obj是不规则形状，这里要获得该不规则区域的最小外接矩形（bounding box），
# 以该最小外接矩形的中心作为obj在dst中的位置
cx = 180
cy = 190
center = (cx, cy)

mask = mask[:,:,0]
result = seamlessClone(src, dst, mask, center, 0)

cv2.imshow('result', result)
while(cv2.waitKey(0) == 27):
    cv2.destroyAllWindows()
    break

#test git

(404, 431, 3) (413, 443, 3) (408, 434, 3)


ValueError: could not broadcast input array from shape (392,412) into shape (0,0)